In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from tqdm import tqdm
import json

# Your session cookie (_t value only, not the full cookie header)
DISCOURSE_COOKIE = "" # "Enter inside quotes"
session = requests.Session()
session.cookies.set("_t", DISCOURSE_COOKIE, domain="discourse.onlinedegree.iitm.ac.in")
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://discourse.onlinedegree.iitm.ac.in",
    "X-Requested-With": "XMLHttpRequest",
    "Cookie": f"_t={DISCOURSE_COOKIE}"
})


BASE_URL = "https://discourse.onlinedegree.iitm.ac.in"

def get_topic_ids(category_slug="courses/tds-kb", category_id=34):
    topics = []
    for page in tqdm(range(0,101)):  # Adjust if you want more pages
        url = f"{BASE_URL}/c/{category_slug}/{category_id}.json?page={page}"
        r = session.get(url)
        if r.status_code != 200:
            break
        data = r.json()
        new_topics = data["topic_list"]["topics"]
        if not new_topics:
            break
        topics.extend(new_topics)
    return topics

def get_topic_ids(category_slug="courses/tds-kb", category_id=34):
    topics = []
    for page in tqdm(range(0, 101)):
        url = f"{BASE_URL}/c/{category_slug}/{category_id}.json?page={page}"
        r = session.get(url)
        print(f"[DEBUG] GET {url} -> {r.status_code}")
        if r.status_code != 200:
            print(f"[ERROR] HTTP {r.status_code}: {r.text[:300]}")
            break
        try:
            data = r.json()
        except Exception as e:
            print(f"[ERROR] JSON parse error: {e}")
            print(r.text[:300])
            break
        topic_list = data.get("topic_list", {})
        new_topics = topic_list.get("topics", [])
        print(f"[DEBUG] Page {page}: {len(new_topics)} topics")
        if not new_topics:
            print("[INFO] No topics found — stopping early.")
            break
        for t in new_topics:
            print(f"  • {t.get('id')} - {t.get('title')}")
        topics.extend(new_topics)
    return topics

def get_posts_in_topic(topic_id):
    url = f"{BASE_URL}/t/{topic_id}.json"
    r = session.get(url)
    if r.status_code != 200:
        print(f"[ERROR] Failed to fetch topic {topic_id}: HTTP {r.status_code}")
        return []

    try:
        data = r.json()
    except Exception as e:
        print(f"[ERROR] JSON parse error for topic {topic_id}: {e}")
        return []

    topic_title = data.get("title", "")
    posts = data.get("post_stream", {}).get("posts", [])

    result = []
    for post in posts:
        content_text = BeautifulSoup(post["cooked"], "html.parser").get_text(separator="\n")
        result.append({
            "topic_id": topic_id,
            "post_number": post["post_number"],
            "username": post["username"],
            "created_at": post["created_at"],
            "content": content_text.strip(),
            "post_url": f"{BASE_URL}/t/{topic_id}/{post['post_number']}",
            "topic_title": topic_title
        })

    return result


all_posts = []
topics = get_topic_ids()

for topic in tqdm(topics):
    # Parse created_at as timezone-aware datetime (UTC)
    created_at = datetime.fromisoformat(topic["created_at"].replace("Z", "+00:00"))
    # Compare with a timezone-aware datetime for Jan 1, 2025 UTC
    if created_at >= datetime(2025, 1, 1, tzinfo=timezone.utc):
        posts = get_posts_in_topic(topic["id"])
        all_posts.extend(posts)

# Save the scraped posts into a JSON file
with open("tds_discourse_posts.json", "w", encoding="utf-8") as f:
    json.dump(all_posts, f, indent=2, ensure_ascii=False)

print(f"Scraped {len(all_posts)} posts.")


  1%|          | 1/101 [00:00<01:29,  1.11it/s]

[DEBUG] GET https://discourse.onlinedegree.iitm.ac.in/c/courses/tds-kb/34.json?page=0 -> 200
[DEBUG] Page 0: 30 topics
  • 178881 - GA4 - Data Sourcing - Discussion Thread [TDS May 2025]
  • 67216 - TDS | References | Guidelines
  • 23335 - About the Tools in Data Science category
  • 176077 - Project1 - Virtual TA - Discussion Thread [TDS May 2025]
  • 179421 - TDS -Project 1 Virtual TA - Course Content Scraping - Clarification
  • 179210 - Project 1 of Tools in Data Science
  • 179246 - TDS P1 help needed
  • 179348 - Promptfoo yaml not fetching results
  • 179153 - Help required in Project 1 - scrapping Discourse data
  • 179311 - IITM Github account flagged
  • 179292 - Unable to deploy api on render
  • 179290 - Pipe tokens quota refill
  • 175592 - GA3 - Large Language Models - Discussion Thread [TDS May 2025]
  • 179103 - TDS Project 1 2025
  • 179213 - Image Size issue with the railway.app deployment made
  • 179200 - Output from the curl command of individual question is diffe

  2%|▏         | 2/101 [00:01<01:01,  1.62it/s]

[DEBUG] GET https://discourse.onlinedegree.iitm.ac.in/c/courses/tds-kb/34.json?page=1 -> 200
[DEBUG] Page 1: 30 topics
  • 178649 - Guide for tds project 1
  • 178166 - How to find API key for discourse page
  • 154294 - TDS project 1,I face ambiguati to make this project,how can I start?
  • 177878 - Marks of GA1
  • 175382 - Lost data in laptop while working on Git bash
  • 177666 - Https://exam.sanand.workers.dev/project-tds-virtual-ta-promptfoo.yaml is not opened
  • 161071 - Which subject to choose in jan term
  • 173525 - GA2 - Deployment Tools - Discussion Thread [TDS May 2025]
  • 163247 - GA3 - Large Language Models - Discussion Thread [TDS Jan 2025]
  • 176823 - Error for Vector Databases (1 mark)
  • 176644 - Project 1 deadline extension
  • 176851 - Subject: Request for Clarification on Assignment Submission Status (GA1, GA2, GA3)
  • 173524 - GA1 - Development Tools - Discussion Thread [TDS May 2025]
  • 176754 - How to manage tds?
  • 176789 - Vercel deployement error
  •

  3%|▎         | 3/101 [00:01<00:52,  1.86it/s]

...

[DEBUG] GET https://discourse.onlinedegree.iitm.ac.in/c/courses/tds-kb/34.json?page=99 -> 200
[DEBUG] Page 99: 30 topics
  • 30385 - Revision Session (Quiz 1)
  • 29803 - Developer Tab in Excel
  • 30118 - Programming Exam
  • 28960 - Web Scrapping , dataframe
  • 27048 - How to check the google drive for extracting pdf from Python
  • 29295 - Auto-correlation plot
  • 29412 - Week-4, Lecture-4, Regression in Excel
  • 29472 - Live session [Jan 21] (Cancellation)
  • 29249 - Cannot import sklearn.externals.six
  • 29456 - Please provide subtitles of lecture videos of week 3
  • 29359 - L5 Forecasting with Python
  • 28784 - Image Labelling
  • 28865 - IMDB Explorer Application - DIY
  • 28454 - Streamlit
  • 28422 - Week 4 Lecture 4.6
  • 27287 - Week 2--extracting pdf
  • 28228 - Week - 3, Lecture 6 - Image Labelling
  • 28119 - Lec 3.3
  • 28046 - L3.6 Image Labelling
  • 27167 - Scraping Order History Data from Amazon
  • 27466 - Scraping the web using Python
  • 27481 - Open refine

100%|██████████| 101/101 [00:46<00:00,  2.15it/s]


[DEBUG] GET https://discourse.onlinedegree.iitm.ac.in/c/courses/tds-kb/34.json?page=100 -> 200
[DEBUG] Page 100: 21 topics
  • 26863 - Every NYT Front Page Since 1852
  • 26806 - Lecture 2.5
  • 26479 - Lecuture 2.3
  • 26410 - Same date for week 1 and week 2 graded assignment
  • 26438 - Week2 lecture 2
  • 26352 - Lecture 2.1, No result coming during google search query
  • 25991 - Scraping web using Python
  • 26294 - How do you open the txt file in Visual Studio Code and display output in JSON format?
  • 24328 - Week 1 lecture 1: Expected Result
  • 26280 - The school performance case-study - question about the primary inference
  • 26049 - Need help with tools in data science
  • 25252 - Stated Problem Vs. Problem
  • 26220 - Week 2 Lecture 2
  • 26221 - BBC weather scraping-URL not converting into JSON
  • 26119 - Ga 2 q7
  • 24799 - 2 Videos for week-1
  • 24249 - Live Sessions
  • 25954 - Imdb scrape
  • 24459 - Reference for more examples on Situation, Problem, Action and Imp

100%|██████████| 3021/3021 [01:46<00:00, 28.32it/s]

Scraped 1150 posts.


In [6]:
r = session.get("https://discourse.onlinedegree.iitm.ac.in/session/current.json")

if r.status_code == 200 and "current_user" in r.json():
    print("✅ Authenticated as:", r.json()["current_user"]["username"])
else:
    print("❌ Not authenticated.")
    print(r.status_code, r.text)


✅ Authenticated as: 21f3001995
